In [46]:
from transformers import (
    AutoTokenizer,
    RobertaModel,
    AutoModelForMaskedLM)
device = 1
generator=AutoModelForMaskedLM.from_pretrained('roberta-large').to(device)

In [47]:
verbalizer_1 = 'World'#'\u0120World'#'\u0120World'#'\u0120good'#'\u0120great' #\u0120wonderful'#'\u0120wonderful'#'\u0120positive'
verbalizer_2 = 'Sports'#'\u0120Sports'#'\u0120Sports' #'\u0120bad'#'\u0120terrible'#'\u0120bad'#'\u0120bad'#'\u0120negative'
verbalizer_3 = 'Business'#'\u0120Business'#'\u0120Business'
verbalizer_4 = 'Tech'#'\u0120Tech'#'\u0120Tech'

In [48]:
tokenizer = AutoTokenizer.from_pretrained('roberta-large')

In [49]:
id_1 = tokenizer.convert_tokens_to_ids(verbalizer_1)
id_2 = tokenizer.convert_tokens_to_ids(verbalizer_2)
id_3 = tokenizer.convert_tokens_to_ids(verbalizer_3)
id_4 = tokenizer.convert_tokens_to_ids(verbalizer_4)

In [50]:
import torch
def evaluate_acc(formatted_prompt, label):
    encoded_input = tokenizer(formatted_prompt, padding='longest', truncation=True, return_tensors='pt', add_special_tokens=True)
    #torch.ne(encoded_input.input_ids, tokenizer.pad_token_id).sum(-1) - 2
    batch_size = len(formatted_prompt)
    seq_len = [1 for _ in range(batch_size)]
    with torch.no_grad():
        logits = generator(
                    input_ids=encoded_input.input_ids.to(device),
                    attention_mask=encoded_input.attention_mask.to(device)
                    ).logits
        #print(logits.shape)
        logits = logits[range(batch_size), seq_len]
        #print(logits.shape)
        logits = logits[:,[id_1, id_2, id_3, id_4]]
        output_probs = torch.softmax(logits,dim=-1)
        predicted_label = torch.argmax(output_probs, dim=-1)
        acc_mat = torch.eq(predicted_label.cpu(), label)
        sum_acc = torch.sum(acc_mat)
    #label = 0
    #for i in range(1,4):
    #    if output_probs[i] > max_prob:
    #        max_prob = output_probs[i]
    #        label = i
    return sum_acc
        

# 1. agnews

In [51]:
# datasets loader
from datasets import load_dataset
agnews_ = load_dataset("ag_news", split = 'test')

Using custom data configuration default
Reusing dataset ag_news (/home/jianyu/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


In [52]:
def prompt_map(example):
    example['text'] = '<mask> News: '+example['text']
    return example

In [60]:
import torch
agnews = agnews_.map(prompt_map)
test_agnews_dataloader = torch.utils.data.DataLoader(agnews, batch_size=128, num_workers=5, pin_memory=True)

Loading cached processed dataset at /home/jianyu/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-e6a4aa55bdc4f43f.arrow


# next(iter(test_agnews_dataloader))['label']

In [ ]:
count = 0# bias, variance also matters a lot. -> more examples
import tqdm # Center staff: 77.88%
for i, batch in tqdm.tqdm(enumerate(iter(test_agnews_dataloader))):# 
    #if i == 0:
    #    continue # �� cabin, 0.8125 | �� Armor, 0.843 | '��py,0.84
    text = batch['text']#element.strip('\n').split('\t')[0] # ��inventoryQuantity, 0.82->
    label = batch['label']#element.strip('\n').split('\t')[-1] # �� guiActiveUnfocused, 0.84375 # Beta版 , Preview版, Rocket版 DragonMagazine版, Preview王, Ultra版, Revolution王
    count += evaluate_acc(text, label)
    #print(label) # MS Why: improved 70.55%->traiing, ellation Reaction->75.73% (comparable-90 on dev), monsming->0.7048
    #if acc == int(label): # 0.7563780568407138, Guide Guide, -> 85/90+ 16-shot
    #    count += 1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

43it [00:25,  1.41it/s]

In [ ]:
count/len(agnews_['text'])

# TODO

agnews_2 = []
agnews_3 = []
agnews_0 = []
agnews_1 = []
for i, text in enumerate(agnews_['train']['text']):
    if len(agnews_0) == 48 and len(agnews_1) == 48 and len(agnews_2) == 48 and len(agnews_3) == 48:
            break
    label = agnews_['train']['label'][i]
    if label == 0:
        if len(agnews_0) == 48:
            continue
        agnews_0.append(text)
    elif label == 1:
        if len(agnews_1) == 48:
            continue
        agnews_1.append(text)
    elif label == 2:
        if len(agnews_2) == 48:
            continue
        agnews_2.append(text)
    elif label == 3:
        if len(agnews_3) == 48:
            continue
        agnews_3.append(text)
    
    

with open('/data/jianyu/test/soft-Q-learning-for-text-generation/24-train.tsv', 'w') as f:
    f.write('text\tlabel\n')
    for text in agnews_0[:24]:
        f.write(text+'\t0\n')
    for text in agnews_1[:24]:
        f.write(text+'\t1\n')
    for text in agnews_2[:24]:
        f.write(text+'\t2\n')
    for text in agnews_3[:24]:
        f.write(text+'\t3\n')

with open('/data/jianyu/test/soft-Q-learning-for-text-generation/24-dev.tsv', 'w') as f:
    f.write('text\tlabel\n')
    for text in agnews_0[24:]:
        f.write(text+'\t0\n')
    for text in agnews_1[24:]:
        f.write(text+'\t1\n')
    for text in agnews_2[24:]:
        f.write(text+'\t2\n')
    for text in agnews_3[24:]:
        f.write(text+'\t3\n')